## Data Pipline 

Preparing individual dataset for merging  

Train and transactions obtained from eda.ipynb 
Holidays, Oil , Items , and stores obtained from  ... 

In [11]:
import pandas as pd
import seaborn as sns 
import numpy as np 
import datetime
import os 
from matplotlib import pyplot as plt
import random
import dask.dataframe as dd
from sklearn import preprocessing

Train

In [12]:
# optimizing size of data types 
dtype_dict={"id":'int32',
            "store_nbr":'int8',
            "item_nbr":'int32',
            "unit_sales":'float32',
            
           }
df1 = pd.read_csv("../../train.csv",dtype=dtype_dict,parse_dates=['date'])

In [13]:
df1['onpromotion'] = df1['onpromotion'].fillna(0)
df1['onpromotion'] = df1['onpromotion'].astype('int8')
df1.dtypes

id                      int32
date           datetime64[ns]
store_nbr                int8
item_nbr                int32
unit_sales            float32
onpromotion              int8
dtype: object

In [144]:
len(df1)

125497040

Transactions 

In [14]:
dtype_dict={"id":'int32',
            "store_nbr":'int8',
            "transactions":'int32',

           }
transactions= pd.read_csv("../../transactions.csv",dtype=dtype_dict,parse_dates=['date'])


In [143]:
transactions.dtypes
len(transactions)

83488

Holidays

In [16]:

holidays = pd.read_csv("../DATA/holidays_clean.csv", parse_dates = ['date'])
holidays = holidays.drop(columns = ['Unnamed: 0'])

In [17]:
# Cleaning holidays table to generate a df with columns date and holiday(bool)

holidays = holidays.drop(holidays[holidays.transferred== True ].index)
holidays = holidays[(holidays.type == 'Holiday') | (holidays.type == 'Bridge')]
holidays = holidays[holidays.locale == 'National']
holidays['holiday'] = 1


In [154]:
#list of national holidays 
holidays = holidays[['date','holiday']]


Oil 

In [145]:
oil = pd.read_csv("../DATA/oil_clean.csv", parse_dates = ['date'])
oil = oil.drop(columns = ['Unnamed: 0'])
len(oil)

1688

Items 

In [146]:
items = pd.read_csv("../../items.csv")
#items = items.drop(columns = ['Unnamed: 0'])
len(items)

4100

Stores

In [147]:
stores = pd.read_csv("../../stores.csv")
#stores = stores.drop(columns = ['Unnamed: 0'])
len(stores)

54

# Sample from the Data 

In [129]:
# sampling 10% of the training dataset
#random.seed(10000)
sample1 = df1.sample(n= round(len(df1)*0.1), random_state=100)
sample1 = sample1.reset_index(drop =True)

In [148]:
len(sample1)

12549704

# Join Tables

In [130]:
# worked well on merging trasnactions with 10 percent of the data 
train_tran = sample1.merge(transactions,on = ['date','store_nbr'],how = 'inner')

In [131]:
train_tran_stores = train_tran.merge(stores,on = "store_nbr",how = "inner")

In [132]:
train_tran_stores_oil = train_tran_stores.merge(oil, on = "date",how = "inner")

In [133]:
train_tran_stores_oil_items = train_tran_stores_oil.merge(items,on = "item_nbr", how = "inner")

In [134]:
#holiday yes no has to be binary 

# holidays[['date','locale']].sort_values(by = 'date')
train_tran_stores_oil_items_hol = train_tran_stores_oil_items.merge(holidays, on = 'date',how = 'left')


# Exploration
-----------------------

In [135]:
print("sample")
print(len(sample1))
print("transactions")
print(len(train_tran))
print("sotores")
print( len(train_tran_stores))
print("oil")
print( len(train_tran_stores_oil))
print("items")
print(len(train_tran_stores_oil_items))
print("hol")
print( len(train_tran_stores_oil_items_hol))


sample
12549704
transactions
12528269
sotores
12528269
oil
12528269
items
12528269
hol
12528269


In [136]:
# doing a left join produces 1 percent more data due to more than one holiday landing on the same day. 
12528238/12549704

0.9982895214102261

Final feature engineering 

In [137]:
train_tran_stores_oil_items_hol['holiday'] = train_tran_stores_oil_items_hol['holiday'].fillna(0)

In [138]:
# this is the output after removing cluster,type,city,state,and class using a sample of 10 percent of the data. 

final_sample = train_tran_stores_oil_items_hol.drop(columns = ['cluster','type','city','state','class'])


In [139]:
# make categorical variables into numerical
# using sklearn encoding
# we need to categorize  store number , item _nbr 
final_sample['item_nbr'].value_counts()

314384     8420
364606     8349
502331     8331
564533     8265
273528     8242
           ... 
1976284       1
2122676       1
2011470       1
2011448       1
2011437       1
Name: item_nbr, Length: 4026, dtype: int64

In [140]:
final_sample

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,oil_price,family,perishable,holiday
0,90594681,2016-09-10,36,1165988,1.0,0,1320,45.315,GROCERY I,0,0.0
1,90536016,2016-09-10,2,1165988,4.0,0,2184,45.315,GROCERY I,0,0.0
2,90558004,2016-09-10,12,1165988,4.0,0,1308,45.315,GROCERY I,0,0.0
3,90593142,2016-09-10,35,1165988,5.0,0,760,45.315,GROCERY I,0,0.0
4,90598928,2016-09-10,38,1165988,7.0,0,1770,45.315,GROCERY I,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
12528264,124856524,2017-08-09,45,2123036,1.0,0,3631,49.590,BEVERAGES,0,0.0
12528265,124327997,2017-08-04,45,2123036,1.0,0,4132,49.570,BEVERAGES,0,0.0
12528266,125055751,2017-08-11,44,2123747,1.0,0,4418,48.810,GROCERY I,0,0.0
12528267,125061343,2017-08-11,46,2123463,2.0,0,3814,48.810,GROCERY I,0,0.0


In [149]:
# scikit learn exploration 
from sklearn.preprocessing import LabelEncoder

In [150]:
final_sample['family_cat'] = LabelEncoder().fit_transform(final_sample['family'])

In [158]:
final_sample

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,oil_price,family,perishable,holiday,family_cat
0,90594681,2016-09-10,36,1165988,1.0,0,1320,45.315,GROCERY I,0,0.0,12
1,90536016,2016-09-10,2,1165988,4.0,0,2184,45.315,GROCERY I,0,0.0,12
2,90558004,2016-09-10,12,1165988,4.0,0,1308,45.315,GROCERY I,0,0.0,12
3,90593142,2016-09-10,35,1165988,5.0,0,760,45.315,GROCERY I,0,0.0,12
4,90598928,2016-09-10,38,1165988,7.0,0,1770,45.315,GROCERY I,0,0.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...
12528264,124856524,2017-08-09,45,2123036,1.0,0,3631,49.590,BEVERAGES,0,0.0,3
12528265,124327997,2017-08-04,45,2123036,1.0,0,4132,49.570,BEVERAGES,0,0.0,3
12528266,125055751,2017-08-11,44,2123747,1.0,0,4418,48.810,GROCERY I,0,0.0,12
12528267,125061343,2017-08-11,46,2123463,2.0,0,3814,48.810,GROCERY I,0,0.0,12


In [159]:
final_sample.to_csv("../DATA/final_sample.csv")